In [72]:
#!pip install -r requirements.txt

In [3]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/lyrics.csv', sep=';', encoding="utf-8")

# Remoção de colunas que não serão utilizadas na análise
df.drop(columns=['id', 'composer', 'album'], inplace=True)
df.head(1)

,title,artist,year,lyric
0,Não Se Esquecerá,Forrozão Tropykália,0,Sei que poderá levar um tempo Sei que poderá d...


### Limpeza dos títulos das músicas

In [122]:
normal_title = df.loc[87, 'title']
normal_title

'De Coração Virado (Morena)'

In [123]:
# igualando todos os títulos para minúsculo
df['title'] = df['title'].str.lower()

# limpeza dos títulos com símbolos
df['title'].replace("\((.*?)\)", '', regex=True, inplace=True)

# removendo espaços nas extremidades
df['title'] = df['title'].str.strip()

#capitalizando título
df['title'] = df['title'].str.title()

In [124]:
processed_title = df.loc[87, 'title']
processed_title

'De Coração Virado'

### Remoção das músicas sem data

In [125]:
df = df[df['year'] > 0]

print(f'Quantidade de músicas com data: {df.title.count()}')

Quantidade de músicas com data: 11046


### Remoção dos registros sem letras

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11046 entries, 2359 to 13404
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   11046 non-null  object
 1   artist  11046 non-null  object
 2   year    11046 non-null  int64 
 3   lyric   10995 non-null  object
dtypes: int64(1), object(3)
memory usage: 431.5+ KB


In [127]:
df = df[~df['lyric'].isnull()]
print(f'Quantidade de registros com letras: {df.lyric.count()}')

Quantidade de registros com letras: 10995


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10995 entries, 2359 to 13404
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   10995 non-null  object
 1   artist  10995 non-null  object
 2   year    10995 non-null  int64 
 3   lyric   10995 non-null  object
dtypes: int64(1), object(3)
memory usage: 429.5+ KB


### Pre-processamento das letras musicais

In [4]:
#df.to_csv('./data/clean_lyrics.csv', index=False)
df = pd.read_csv('./data/clean_lyrics.csv')

#### Remoção de stop words

In [3]:
# carregando dataset de sujeiras encontradas em algumas letras
noise = pd.read_csv('./data/stop_words.txt')
noise = noise['stopwords']

In [4]:
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS

nlp = spacy.load('pt_core_news_sm')

lyric_noise = set(noise.to_list())
stop_words = STOP_WORDS.union(lyric_noise)
stop_words = [word for word in stop_words]

In [16]:
import re

def clean_lyrics(text):
  txt = ''
  #caracteres em minusculo   
  txt = text
  #coletando apenas letras
  words = re.findall('[a-zéóáêâãõç|A-ZÉÓÁÊÂÃÕÇ]+', txt)
  txt = (' ').join(words)
  #remove stop words
  doc = nlp(txt)
  vocab = []
  for token in doc:
      if token.lower_ not in stop_words:
          if token.pos_ == 'VERB':
              vocab.append(token.lemma_.lower())
          elif token.pos_ == 'PROPN':
              vocab.append(token.text)
          else:
              vocab.append(token.lower_)

  txt = ' '.join(vocab)
  
  return txt

In [17]:
## Testando preprocessmaneto
normal_lyric = df.loc[6, 'lyric']
processed_lyric = clean_lyrics(normal_lyric)

print(f'antes: {normal_lyric}')
print(f'depois: {processed_lyric}')


antes: Eu fui dançar Twist lá pra banda do Caju Mas lá pra meia-noite começou um sururu (2x) Iê, iê, iê, iê, iê, que confusão Iê, iê, iê, só sendo a tentação do cão (coro repete) Tudo aconteceu naquela brincadeira, Porque o dono do baile barrou uma feiticeira Que saiu se maldizendo daquela festança E entrou no cemitério pedindo vingança Iê, iê, iê, iê, iê, que confusão Iê, iê, iê, só sendo a tentação do cão (coro repete) Quando eu vi a coisa preta, tratei de correr Mas uma voz diferente não parava de dizer: "Se correr o bicho pega, se ficar o bicho come" Olhei pra minha esquerda, vi um lobisomem Iê, iê, iê, iê, iê, que confusão Iê, iê, iê, só sendo a tentação do cão (coro repete)
depois: dançar Twist banda Caju meia noite começar sururu confusão sendo tentação cão coro acontecer brincadeira dono baile barrar feiticeira sair maldizer festança entrar cemitério pedir vingança confusão sendo tentação cão coro vir preta tratar correr diferente parar correr bicho pego ficar bicho comer olhei

In [18]:
#removendo ruídos das letras
df['clean_lyric'] = df['lyric'].apply(clean_lyrics)

In [19]:
"""
Bertaglia, Thales Felipe Costa, and Maria das Graças Volpe Nunes. 
“Exploring Word Embeddings for Unsupervised Textual User-Generated Content Normalization.” 
Proceedings of the 2nd Workshop on Noisy User-generated Text (WNUT). 2016.
"""
import spacy
nlp = spacy.load('pt_core_news_sm')

from enelvo.normaliser import Normaliser
norm = Normaliser()

def clean_lyrics_2(lyric):
    doc = nlp(lyric)
    normalized = ' '.join(norm.normalise(word.text) for word in doc)
    
    return normalized

In [23]:
processed_lyric = df.loc[0, 'clean_lyric']
print(f'antes: {processed_lyric}')
print(f'depois: {clean_lyrics_2(processed_lyric)}')

antes: discutir teimoso perder precioso viver João Teimoso nome Dorme pé rede bebido fome comida sede Foge meninas boas preferir coroas comedir parar cara cismo cara
depois: discutir teimoso perder precioso viver joão teimoso nome dorme pé rede bebido fome comida sede foge meninas boas preferir coroas comedir parar cara cismo cara


In [2]:
import multiprocessing

workers = multiprocessing.cpu_count() - 1
workers

7

In [24]:
import numpy as np
from multiprocessing import Process

partitions = np.partition(df['clean_lyric'], 3)
print(partitions[0])
    

abala abalar cajueiro abaladora pequenino carregar fulo balanço gaia cajueiro saudade embalançar amor deixei gravar tronco cajueiro nome dela coração jurar ia deixar deixar ficar solidão nome dela fazer cajueiro apagar doido homem confiar mulher voltar


In [45]:
# função para remover palavras que contêm repetições sequenciais
def remove_words_with_reps(text):
  words = text.split()
  for i, word in enumerate(words):
      reps = 0
      for j, character in enumerate(word):
        if j > 0 and character == word[j-1]:
            reps+=1
      if reps > 1:
        words.pop(i)
      
  return (' ').join(words)

In [48]:
#Testando removedor de sequenciais
lyric = df.loc[245, 'clean_lyric']
processed_lyric = remove_words_with_reps(lyric)

print(f'antes: {lyric}')
print(f'depois: {processed_lyric}')

antes: diferença mulher home espere home cabelo peito queixo cabeludo mulher so dia manhã adão comer maçã eva comer ficar adão eva adão dar mancada eva dar mulher perna braço coxo nariz boca muita inteligência bicho homem jeito reparar direito pouquinho diferença breque achar diferença sapato homem agacho
depois: diferença mulher home espere home cabelo peito queixo cabeludo mulher so dia manhã adão comer maçã eva comer ficar adão eva adão dar mancada eva dar mulher perna braço coxo nariz boca muita inteligência bicho homem jeito reparar direito pouquinho diferença breque achar diferença sapato homem agacho


In [157]:
df['clean_lyric'] = df['clean_lyric'].apply(remove_words_with_reps)

#### criando decadas

In [162]:
#função para criar decadas
decade = lambda year: int(((((year - (year % 10)) / 10) % 10) * 10))

In [163]:
#gerando decadas
df['decade'] = df['year'].apply(decade)

#### gerando colunas de palavras únicas

In [164]:
#função para retornar palavras unicas
unique = lambda text: set(text.split())

In [165]:
df['unique_words'] = df['clean_lyric'].apply(unique)

#### Passando letras para listas

In [166]:
#função para retornar palavras em formato de lista
to_words = lambda text: text.split()

In [167]:
# transformando letras em lista de palavras
df['words'] = df['clean_lyric'].apply(to_words)

In [169]:
#armazenando
df.to_csv('./data/clean_lyrics.csv', index=False)